In [5]:
import pandas as pd
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

# 1. Загрузка данных
data = sns.load_dataset('titanic')

# 2. Определение признаков
numeric_features = ['age', 'fare']
categorical_features = ['sex', 'embarked', 'pclass', 'sibsp', 'parch']

# 3. Создание Pipeline для обработки данных
preprocess_pipeline = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numeric_features),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_features)
    ]
)


# 4. Разделение данных на признаки и целевую переменную
X = data.drop('survived', axis=1)
y = data['survived']

# 5. Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 6. Применение Pipeline к данным
X_train_processed = preprocess_pipeline.fit_transform(X_train)
X_test_processed = preprocess_pipeline.transform(X_test)

# 7. Построение модели
model = Pipeline([
    ('classifier', RandomForestClassifier())
])

# 8. Подбор гиперпараметров
param_grid = {
    'classifier__n_estimators': [10, 50, 100, 200, 300],
    'classifier__max_depth': [3, 5, 10, 15]
}

grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X_train_processed, y_train)

# 9. Оценка модели
print(f"Лучшие параметры: {grid_search.best_params_}")
print(f"Лучшее качество: {grid_search.best_score_}")
print(f"best_index_: {grid_search.best_index_}")


Лучшие параметры: {'classifier__max_depth': 5, 'classifier__n_estimators': 50}
Лучшее качество: 0.8159657244164287
best_index_: 6
